# Data Analytics Tech Challenge - Fase 2 Machine Learning and Data Analytics

## Grupo 41

- [André Guilherme Corrêa](https://www.linkedin.com/in/agcorrea/)
- [Letícia Maia de Oliveira](https://www.linkedin.com/in/lemaia/)
- [Nathalia Souza Vaz](https://www.linkedin.com/in/sousanathalias/)
- [Renan Felipe Silva](https://www.linkedin.com/in/renan-silva-16960313a)

## O Problema

Imagine que você foi escalado para um time de investimentos e precisará realizar um modelo preditivo com dados da IBOVESPA (Bolsa de valores) para criar uma série temporal e prever diariamente o fechamento da base.

Para isso utilize a base de dados contida no [site da investing](https://br.investing.com/indices/bovespa-historical-data) e selecione o período “diário”, com o intervalo de tempo que achar adequado.

Você precisará demonstrar para o time de investimentos:

1. O modelo com o storytelling, desde a captura do dado até a entrega do modelo;
2. Justificar a técnica utilizada;
3. Atingir uma acuracidade adequada (acima de 70%).

> DICA: Utilize o maior intervalo de tempo possível para atingir maior acuracidade no modelo preditivo.

## Dicionário de dados

- Data:
- Último: preço do índice no fechamento
- Abertura: preço do índice na abertura do mercado
- Máxima: maior preço durante o dia
- Mínima: menor preço durante o dia
- Vol.: volume de negociações
- Var%: variação do preço entre a abertura e o fechamento

In [1]:
import pandas as pd
import numpy as np

## Captura dos dados

### Importação inicial

In [2]:
# Lendo dados da fonte externa para que todos consigam rodar o código no Colab sem ter que fazer upload do arquivo
# Mantendo Dataframe original para facilitar rodar as transformações multiplas vezes
df_raw = pd.read_csv(
    'https://raw.githubusercontent.com/agcorrea/fiap-techchallenge-grupo41/main/fase-2/dados/ibovespa_2003-01-01_2023-03-09.csv',
    parse_dates=['Data'], dayfirst=True,
    thousands='.', decimal=',')
df_raw.head()

,Data,Último,Abertura,Máxima,Mínima,Vol.,Var%
0,2023-03-09,105071,106540,106724,105053,"19,17M","-1,38%"
1,2023-03-08,106540,104228,106721,104228,"15,90M","2,22%"
2,2023-03-07,104228,104700,105179,103480,"12,58M","-0,45%"
3,2023-03-06,104700,103865,105171,103170,"14,28M","0,80%"
4,2023-03-03,103866,103326,104440,103323,"13,10M","0,52%"


In [3]:
df_raw.dtypes

Data        datetime64[ns]
Último               int64
Abertura             int64
Máxima               int64
Mínima               int64
Vol.                object
Var%                object
dtype: object

As colunas `Data`, `Último`, `Abertura`, `Máxima` e `Mínima` foram corretamente parseadas com os parâmetros do `pd.read_csv`.

Teremos que tratar as colunas `Vol.` e `Var%` para que se tornem númericas.

### Transformações

No arquivo CSV baixado do site investing.com, os valores das colunas `Vol.` e `Var%` estão formatados como texto.



In [4]:
# Explorando quais unidades estão presentes na coluna Vol.
df_raw['Vol.'].str.replace('\d', '', regex=True).unique()

array([',M', ',K', nan], dtype=object)

In [5]:
f"Porcentagem de valores da coluna `Var%` que terminam com %: {len(df_raw[df_raw['Var%'].str.endswith('%')]) / len(df_raw) * 100}%"

'Porcentagem de valores da coluna `Var%` que terminam com %: 100.0%'


O código à seguir transforma as colunas para seus respectivos valores numéricos.

*   `Vol.`: os valores estão representados com os símbolos M e K que significam milhões e milhares, respectivamente.
*   `Var%`: os valores estão representados com o símbolo de porcentagem e separador decimal `,`.



In [6]:
# Função para transformar a coluna `Vol.` de texto para números inteiros
def parse_vol(x):
  # Não faz nada caso não seja uma string
  if type(x) != str:
    return None

  # Multiplica por 1 milhão se possuí símbolo M ou por 1 mil se símbolo K
  multiplier = 1_000_000 if x.endswith('M') else 1_000

  # Remove o símbolo M ou K
  result = x[:-1]

  # Altera o separador para . ao invés de ,
  result = result.replace(',', '.')
  # Transforma a string em número
  result = float(result)

  # Multiplica o número pelo multiplicador e transforma em inteiro
  return int(result * multiplier)

In [7]:
# Função para transformar a coluna `Var%` de texto para número decimal
def parse_var(x):
  # Remove o símbolo de %
  result = x[:-1]

  # Altera o separador para . ao invés de ,
  result = result.replace(',', '.')

  # Transforma a string em número
  result = float(result)

  # Transforma em porcentagem
  return result / 100

In [8]:
# Cria um novo DataFrame a partir dos dados crus para aplicar as transformações
df = df_raw.copy()
df['Vol.'] = df['Vol.'].apply(parse_vol)
df['Var%'] = df['Var%'].apply(parse_var)
df = df.set_index('Data', drop=True)
df.head()

,Último,Abertura,Máxima,Mínima,Vol.,Var%
Data,,,,,,
2023-03-09,105071,106540,106724,105053,19170000.0,-0.0138
2023-03-08,106540,104228,106721,104228,15900000.0,0.0222
2023-03-07,104228,104700,105179,103480,12580000.0,-0.0045
2023-03-06,104700,103865,105171,103170,14280000.0,0.0080
2023-03-03,103866,103326,104440,103323,13100000.0,0.0052


In [9]:
df.dtypes

Último        int64
Abertura      int64
Máxima        int64
Mínima        int64
Vol.        float64
Var%        float64
dtype: object

Após as transformações, todos os tipos estão corretos.

### Dados faltantes

Neste bloco, os dados faltantes serão encontrados e tratados caso necessário.

In [10]:
# Quantidade de valores nulos por coluna
df.isnull().sum()

Último      0
Abertura    0
Máxima      0
Mínima      0
Vol.        1
Var%        0
dtype: int64

Temos um único valor nulo na coluna `Vol.`.

In [11]:
# Filtra dataframe procurando por valores nulos na coluna Vol.
df[df.isnull()['Vol.'] == True]

,Último,Abertura,Máxima,Mínima,Vol.,Var%
Data,,,,,,
2016-02-10,40377,40592,40592,39960,NaN,-0.0053


Segundo o site [Yahoo! Finance](https://yhoo.it/3P3HaYv), no dia 10/02/2016 ocorreu um volume de movimentações de 3.70M.

![ibovespa_2016-02-10.png](https://raw.githubusercontent.com/agcorrea/fiap-techchallenge-grupo41/main/fase-2/assets/ibovespa_2016-02-10.png)

In [20]:
# Atualiza o valor nulo com o encontrado no Yahoo! Finance
df.loc['2016-02-10', 'Vol.'] = 3.7 * 1_000_000
df[df.index == '2016-02-10']

,Último,Abertura,Máxima,Mínima,Vol.,Var%
Data,,,,,,
2016-02-10,40377,40592,40592,39960,3700000.0,-0.0053


In [22]:
print('Teste de sanidade, nenhum valor deve ser nulo:')

# Quantidade de valores nulos por coluna
df.isnull().sum()

Teste de sanidade, nenhum valor deve ser nulo:


Último      0
Abertura    0
Máxima      0
Mínima      0
Vol.        0
Var%        0
dtype: int64

## Análise Exploratória dos Dados

## Modelo Preditivo